In [63]:
import pandas as pd
import requests
from functools import cache

In [64]:
URL = 'https://api.mapy.cz/v1/rgeocode'

API_KLIC = 'eyJpIjoyNTcsImMiOjE2Njc0ODU2MjN9.c_UlvdpHGTI_Jb-TNMYlDYuIkCLJaUpi911RdlwPsAY'

LANG = 'cs'

HEADERS = {
"Origin": "https://pro.mapy.cz",
"Referer": "https://pro.mapy.cz/",
"Sec-Fetch-Dest": "empty",
"Sec-Fetch-Mode": "cors",
"Sec-Fetch-Site": "same-site",
"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
}

In [65]:
@cache
def rgeo(lon: float, lat: float) -> tuple[str, str]: 
    r = requests.get(url, headers=headers, params={
        'lang': lang,
    'lon': lon,
    'lat': lat,
    'apikey': api_klic,
    })
    # print(r.status_code, r.reason)
    data = r.json()

    return data

In [66]:
df = pd.read_csv("novy_grid.csv", encoding="utf-8")

In [67]:
df.head()

,distance,latitude,longitude,geometry,id_cukrarna,distance_mhd,name_mhd
0,2.400607,49.9468,14.3894,POINT (14.3894 49.9468),271,0.567415,"Jíloviště,Výzkumný ústav"
1,2.548011,49.9468,14.3944,POINT (14.3944 49.9468),271,0.128849,Strnady
2,4.010847,49.9518,14.3394,POINT (14.3394 49.9518),589,0.179248,Kazín
3,3.769092,49.9518,14.3444,POINT (14.3444 49.9518),589,0.405534,Kazín
4,3.597397,49.9518,14.3494,POINT (14.3494 49.9518),589,0.936626,Ke Kazínu


In [68]:
rgeo(14.3894, 49.9468)

{'items': [{'name': 'Praha-Zbraslav',
   'label': 'Městská část',
   'position': {'lon': 14.3799, 'lat': 49.97403},
   'type': 'regional.municipality_part',
   'location': 'Praha, Česko',
   'regionalStructure': [{'name': 'Praha-Zbraslav',
     'type': 'regional.municipality_part'},
    {'name': 'Praha', 'type': 'regional.municipality'},
    {'name': 'okres Hlavní město Praha', 'type': 'regional.region'},
    {'name': 'kraj Hlavní město Praha', 'type': 'regional.region'},
    {'name': 'Česko', 'type': 'regional.country', 'isoCode': 'CZ'}]}],
 'location_boxes': {}}

In [76]:
def rgeo_to_name_and_zip(data):
    name, street, number = None, None, None
    for i in data['items'][0]['regionalStructure']:
        if i['type'] == 'regional.municipality_part':
            name = i['name'].replace('Praha-', '')
        if i['type'] == 'regional.street':
            street = i['name']
        if i['type'] == 'regional.address':
            number = i['name']
    # if 'zip' in data['items'][0]:        
    #     _zip = data['items'][0]['zip']
    return name, street, number

In [73]:
# df = df[:100]

In [77]:
df["rgeo_data"] = df.apply(lambda row: rgeo(row.longitude, row.latitude), axis=1)
df[["name", "street", "number"]] = df["rgeo_data"].apply(rgeo_to_name_and_zip).apply(pd.Series)
df.drop(columns=["rgeo_data"], inplace=True)

In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2490 entries, 0 to 2489
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   distance      2490 non-null   float64
 1   latitude      2490 non-null   float64
 2   longitude     2490 non-null   float64
 3   geometry      2490 non-null   object 
 4   id_cukrarna   2490 non-null   int64  
 5   distance_mhd  2490 non-null   float64
 6   name_mhd      2490 non-null   object 
 7   name          2191 non-null   object 
 8   street        1679 non-null   object 
 9   number        1407 non-null   object 
dtypes: float64(4), int64(1), object(5)
memory usage: 194.7+ KB


In [ ]:
df.to_csv("kompletni_data.csv")